# Introduction

This notebook contains a walkthrough of the memory optimization for the train dataset of this Kaggle's competition. The main goal is to reduce as much as posible the memory consumption of the dataset without losing any information by using the correct data types for each column.  

> Note that instead of pandas you can also use polars to load the data faster.

# Setup

In [1]:
%pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Path to files
test_csv_path = "./data/test.csv"
train_csv_path = "./data/train.csv"
train_labels_csv = "./data/train_labels.csv"

sample_submission_csv = "./data/sample_submission.csv"

> Remember to check the data dictionary to get a better understanding of the data.

Since the dataset is quite large, we'll start by loading only a subset. This will allow us to perform the memory optimization faster.

In [4]:
# Read a subset of the train dataset
train_df = pd.read_csv(train_csv_path, index_col="index", nrows=10_000)
train_df.head()

,session_id,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
index,,,,,,,,,,,,,,,,,,,
0,20090312431273200,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


# Memory optimization

In [5]:
# Check the shape of the train dataset
train_df.shape

(10000, 19)

In [6]:
# Check the memory usage of the train dataset subset
train_df.memory_usage(deep=True)

Index              80000
session_id         80000
elapsed_time       80000
event_name        700443
name              638871
level              80000
page               80000
room_coor_x        80000
room_coor_y        80000
screen_coor_x      80000
screen_coor_y      80000
hover_duration     80000
text              525518
fqid              567025
room_fqid         847275
text_fqid         569413
fullscreen         80000
hq                 80000
music              80000
level_group       614050
dtype: int64

In [7]:
# Check the memory usage of the train dataset subset in mb
train_df.memory_usage(deep=True).sum() / 1024**2

5.247683525085449

In [8]:
# Columns names
train_df.columns

Index(['session_id', 'elapsed_time', 'event_name', 'name', 'level', 'page',
       'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y',
       'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid',
       'fullscreen', 'hq', 'music', 'level_group'],
      dtype='object')

In [9]:
# Check the data types of the train dataset subset
train_df.dtypes

session_id          int64
elapsed_time        int64
event_name         object
name               object
level               int64
page              float64
room_coor_x       float64
room_coor_y       float64
screen_coor_x     float64
screen_coor_y     float64
hover_duration    float64
text               object
fqid               object
room_fqid          object
text_fqid          object
fullscreen          int64
hq                  int64
music               int64
level_group        object
dtype: object

In [10]:
# Number of nan values in each column
train_df.isna().sum()

session_id           0
elapsed_time         0
event_name           0
name                 0
level                0
page              9766
room_coor_x       1015
room_coor_y       1015
screen_coor_x     1015
screen_coor_y     1015
hover_duration    9013
text              6368
fqid              3031
room_fqid            0
text_fqid         6368
fullscreen           0
hq                   0
music                0
level_group          0
dtype: int64

In [11]:
# Number of unique values in each column
train_df.nunique()

session_id          10
elapsed_time      9980
event_name          11
name                 6
level               23
page                 7
room_coor_x       8669
room_coor_y       7243
screen_coor_x     1011
screen_coor_y      704
hover_duration     490
text               522
fqid               118
room_fqid           19
text_fqid          104
fullscreen           2
hq                   2
music                2
level_group          3
dtype: int64

Since some columns like `fullscreen`, `hq` and `music` are binary values (0, 1), we can use the `bool` type instead of `int64`.

In [12]:
train_df["fullscreen"] = train_df["fullscreen"].astype("bool")
train_df["hq"] = train_df["hq"].astype("bool")
train_df["music"] = train_df["music"].astype("bool")

In [13]:
# Check the event_name column
train_df["event_name"].value_counts()

event_name
navigate_click        4152
person_click          2256
cutscene_click        1035
object_click           736
map_hover              539
object_hover           448
notification_click     262
notebook_click         234
map_click              231
observation_click       79
checkpoint              28
Name: count, dtype: int64

In [14]:
# Check the name column
train_df["name"].value_counts()

name
basic        4855
undefined    4745
close         291
open          103
prev            4
next            2
Name: count, dtype: int64

The `event_name` and `name` columns are categorical values, so we can use the `category` type instead of `object`.

In [15]:
train_df["event_name"] = train_df["event_name"].astype("category")
train_df["name"] = train_df["name"].astype("category")

In [16]:
# Check the level column range of values
train_df["level"].min(), train_df["level"].max()

(0, 22)

In [17]:
# Since the range of the level column is small, we can convert it to int8
train_df["level"] = train_df["level"].astype("int8")

In [18]:
# Check the page column
train_df["page"].value_counts()

page
6.0    57
5.0    44
1.0    38
4.0    35
0.0    31
3.0    22
2.0     7
Name: count, dtype: int64

As with the `level` column, we can use a smaller integer type for it. In this case, we'll use the `Int8` due to the presence of `NaN` values.

In [19]:
train_df["page"] = train_df["page"].astype("Int8")

In the data dictionary we can see that the `level_group` is a categorical value, so we can use the `category` type. But since the order in this case is importart, first we'll create a new category type with the correct order and then we'll use it to convert the column.

In [20]:
level_group_cat_type = pd.CategoricalDtype(
    categories=["0-4", "5-12", "13-22"], ordered=True
)
train_df["level_group"] = train_df["level_group"].astype(level_group_cat_type)

Now, let's perform a quick check of some columns like `fqid`, `room_fqid`, etc.

In [21]:
train_df["fqid"].value_counts()

fqid
worker                          661
gramps                          451
archivist                       421
toentry                         307
wells                           278
                               ... 
reader.paper0.prev                2
reader.paper2.prev                2
tocloset                          2
reader.paper1.prev                1
journals_flag.pic_1_old.next      1
Name: count, Length: 118, dtype: int64

In [22]:
train_df["room_fqid"].value_counts()

room_fqid
tunic.historicalsociety.entry              1517
tunic.wildlife.center                      1055
tunic.historicalsociety.cage                786
tunic.library.frontdesk                     772
tunic.historicalsociety.stacks              705
tunic.historicalsociety.frontdesk           703
tunic.historicalsociety.closet_dirty        603
tunic.humanecology.frontdesk                499
tunic.kohlcenter.halloffame                 479
tunic.historicalsociety.basement            439
tunic.drycleaner.frontdesk                  367
tunic.historicalsociety.collection          354
tunic.historicalsociety.closet              339
tunic.library.microfiche                    327
tunic.flaghouse.entry                       313
tunic.capitol_2.hall                        274
tunic.capitol_1.hall                        191
tunic.historicalsociety.collection_flag     176
tunic.capitol_0.hall                        101
Name: count, dtype: int64

In [23]:
train_df["text_fqid"].value_counts()

text_fqid
tunic.historicalsociety.cage.confrontation                     212
tunic.historicalsociety.entry.groupconvo                       184
tunic.wildlife.center.crane_ranger.crane                       164
tunic.historicalsociety.frontdesk.archivist.newspaper          164
tunic.historicalsociety.frontdesk.archivist.have_glass         156
                                                              ... 
tunic.capitol_2.hall.chap4_finale_c                              1
tunic.capitol_1.hall.boss.writeitup                              1
tunic.capitol_1.hall.chap2_finale_c                              1
tunic.humanecology.frontdesk.block_0                             1
tunic.historicalsociety.frontdesk.archivist.newspaper_recap      1
Name: count, Length: 104, dtype: int64

As we can see, we can convert this columns to a category type without the loss of any information.

In [24]:
train_df["fqid"] = train_df["fqid"].astype("category")
train_df["room_fqid"] = train_df["room_fqid"].astype("category")
train_df["text_fqid"] = train_df["text_fqid"].astype("category")

Columns like `room_coor_x`, `room_coor_y`, `screen_coor_x`, `screen_coor_y` and `hover_duration` have a float64 type but looking at the data we can see that maybe they could be converted to float32 without a loss in information.

In [25]:
train_df["room_coor_x"].max()

1234.4698376483232

In [26]:
train_df["room_coor_y"].max()

484.99533347049936

In [27]:
train_df["screen_coor_x"].max()

1196.0

In [28]:
train_df["screen_coor_y"].max()

898.0

In [29]:
train_df["hover_duration"].max()

26695.0

None of the columns needs the precision of a `float64`, so we can convert them to `float32` without losing any information.

In [30]:
train_df["room_coor_x"] = train_df["room_coor_x"].astype("float32")
train_df["room_coor_y"] = train_df["room_coor_y"].astype("float32")
train_df["screen_coor_x"] = train_df["screen_coor_x"].astype("float32")
train_df["screen_coor_y"] = train_df["screen_coor_y"].astype("float32")
train_df["hover_duration"] = train_df["hover_duration"].astype("float32")

The `elapsed_time` columns contains very large values, so we will leave it as an `int64` type. Also, we have to remember that we are working with a subset of the data, so the values in the full dataset could be even larger.

Finally, we can convert the `text` columns to a category type.

In [31]:
train_df["text"] = train_df["text"].astype("str")

Finally, let's check the memory usage again.

In [32]:
train_df.memory_usage(deep=True)

Index              80000
session_id         80000
elapsed_time       80000
event_name         11068
name               10545
level              10000
page               20000
room_coor_x        40000
room_coor_y        40000
screen_coor_x      40000
screen_coor_y      40000
hover_duration     40000
text              703822
fqid               22548
room_fqid          12160
text_fqid          24655
fullscreen         10000
hq                 10000
music              10000
level_group        10291
dtype: int64

In [33]:
# Check the memory usage of the train dataset subset in mb
train_df.memory_usage(deep=True).sum() / 1024**2

1.235093116760254

We have reduced the memory usage drastically by just converting some columns to categorical types and the numerical values to more efficient types for the data ranges they contain. This will allow us to load the full dataset later more efficiently and to be able to iterate faster over the data when needed.  

# Load the full dataset

In [34]:
train_df = pd.read_csv(
    train_csv_path,
    index_col="index",
    dtype={
        "session_id": "int64",
        "elapsed_time": "int64",
        "event_name": "category",
        "name": "category",
        "level": "int8",
        "page": "Int8",
        "room_coor_x": "float32",
        "room_coor_y": "float32",
        "screen_coor_x": "float32",
        "screen_coor_y": "float32",
        "hover_duration": "float32",
        "text": "str",
        "fqid": "category",
        "room_fqid": "category",
        "text_fqid": "category",
        "fullscreen": "bool",
        "hq": "bool",
        "music": "bool",
        "level_group": level_group_cat_type,
    },
)

In [35]:
# Check the memory usage of the full train dataset in mb
train_df.memory_usage(deep=True).sum() / 1024**2

2749.636650085449

Note that instead of using `bool` for the binary columns, we can also use `int8`.